<a href="https://colab.research.google.com/github/EtzionR/NLP4GeoAI/blob/main/preprocess_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --force-reinstall datasets==2.18.0

  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pyarrow-22.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.2 kB)
  Using cached pyarrow_hotfix-0.7-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.6.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.18-py312-none-any.whl.metadata (7.5 kB)
  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached aiohttp-3.13.2-cp312-cp312-manylinux20

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np



In [51]:
STOPWORDS_START = {"'s", "n't",  "'re","'m", "'ll", "'ve","'ll",
                   '$',';', '%',
                   '?','/?', ':',
                   ', ',"''",
                   '!', '!!', '!!!',
                   '/.', '.','...','/-', '/', '..'}

STOPWORDS_END = {'``'}

STOPWORDS_BOTH = {'"','--','-'}

len(STOPWORDS_START)+len(STOPWORDS_BOTH)+len(STOPWORDS_END)

27

In [12]:
CONLL_LOC = 5
ONTONOTES_LOC = {7, 20, 23}

In [44]:
from datasets import load_dataset
ontonotes = load_dataset("tner/ontonotes5")
ontonotes

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 59924
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 8528
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 8262
    })
})

In [43]:
from datasets import load_dataset

conll = load_dataset("eriktks/conll2003")
conll

/usr/local/lib/python3.12/dist-packages/datasets/load.py:1461: FutureWarning: The repository for eriktks/conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eriktks/conll2003
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [54]:
def dataset_to_pandas(ds):
    return pd.concat([ds[key].to_pandas() for key in ds.keys()])


def tokens_to_text(tokens):
    text = ' '.join(tokens)
    for sw in STOPWORDS_START:
        text = text.replace(f' {sw}', sw)
    for sw in STOPWORDS_END:
        text = text.replace(f'{sw} ', sw)
    for sw in STOPWORDS_BOTH:
        text = text.replace(f' {sw} ', sw)
    return text

cdf = dataset_to_pandas(conll)
cdf['Text'] = cdf.tokens.apply(tokens_to_text)
cdf['Source'] = ['conll2003']*len(cdf)
cdf['AsLOC'] = cdf.ner_tags.apply(lambda lst: CONLL_LOC in lst)
cdf

,id,tokens,pos_tags,chunk_tags,ner_tags,Text,Source,AsLOC
0,0,"[EU, rejects, German, call, to, boycott, Briti...","[22, 42, 16, 21, 35, 37, 16, 21, 7]","[11, 21, 11, 12, 21, 22, 11, 12, 0]","[3, 0, 7, 0, 0, 0, 7, 0, 0]",EU rejects German call to boycott British lamb.,conll2003,False
1,1,"[Peter, Blackburn]","[22, 22]","[11, 12]","[1, 2]",Peter Blackburn,conll2003,False
2,2,"[BRUSSELS, 1996-08-22]","[22, 11]","[11, 12]","[5, 0]",BRUSSELS 1996-08-22,conll2003,True
3,3,"[The, European, Commission, said, on, Thursday...","[12, 22, 22, 38, 15, 22, 28, 38, 15, 16, 21, 3...","[11, 12, 12, 21, 13, 11, 11, 21, 13, 11, 12, 1...","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, ...",The European Commission said on Thursday it di...,conll2003,False
4,4,"[Germany, 's, representative, to, the, Europea...","[22, 27, 21, 35, 12, 22, 22, 27, 16, 21, 22, 2...","[11, 11, 12, 13, 11, 12, 12, 11, 12, 12, 12, 1...","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, ...",Germany's representative to the European Union...,conll2003,True
...,...,...,...,...,...,...,...,...
3448,3448,"[That, is, why, this, is, so, emotional, a, ni...","[12, 42, 46, 12, 42, 30, 16, 12, 21, 15, 28, 6...","[11, 21, 3, 11, 21, 1, 2, 11, 12, 13, 11, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",That is why this is so emotional a night for m...,conll2003,False
3449,3449,"["", It, was, the, joy, that, we, all, had, ove...","[0, 28, 38, 12, 21, 15, 28, 12, 38, 15, 12, 21...","[0, 11, 21, 11, 12, 17, 11, 11, 21, 13, 11, 12...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",""" It was the joy that we all had over the peri...",conll2003,False
3450,3450,"[Charlton, managed, Ireland, for, 93, matches,...","[22, 38, 22, 15, 11, 24, 6, 15, 43, 21, 28, 38...","[11, 21, 11, 13, 11, 12, 0, 13, 11, 12, 11, 21...","[1, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Charlton managed Ireland for 93 matches, durin...",conll2003,True
3451,3451,"[He, guided, Ireland, to, two, successive, Wor...","[28, 38, 22, 35, 11, 16, 22, 22, 24, 24, 10, 3...","[11, 21, 11, 21, 11, 12, 12, 12, 12, 12, 0, 13...","[0, 0, 5, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 7, ...",He guided Ireland to two successive World Cup ...,conll2003,True


In [55]:
odf = dataset_to_pandas(ontonotes)
odf['Text'] = odf.tokens.apply(tokens_to_text)
odf['Source'] = ['ontonotes5']*len(odf)
odf['AsLOC'] = odf.tags.apply(lambda lst: sum([i in ONTONOTES_LOC for i in lst])>0)
odf

,tokens,tags,Text,Source,AsLOC
0,"[People, start, their, own, businesses, for, m...","[0, 0, 0, 0, 0, 0, 0, 0, 0]",People start their own businesses for many rea...,ontonotes5,False
1,"[But, a, chance, to, fill, out, sales, -, tax,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",But a chance to fill out sales-tax records is ...,ontonotes5,False
2,"[Red, tape, is, the, bugaboo, of, small, busin...","[0, 0, 0, 0, 0, 0, 0, 0, 0]",Red tape is the bugaboo of small business.,ontonotes5,False
3,"[Ironically, ,, the, person, who, wants, to, r...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Ironically, the person who wants to run his or...",ontonotes5,False
4,"[Yet, every, business, owner, has, to, face, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Yet every business owner has to face the mound...,ontonotes5,False
...,...,...,...,...,...
8257,[okay],[0],okay,ontonotes5,False
8258,"[and, once, you, 're, in, Holland, the, line, ...","[0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0]",and once you're in Holland the line will be re...,ontonotes5,True
8259,"[I, mean, the, phones, here, are, pretty, good...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",I mean the phones here are pretty good. once y...,ontonotes5,False
8260,"[It, 's, just, these, to, America]","[0, 0, 0, 0, 0, 7]",It's just these to America,ontonotes5,True


In [58]:
df = pd.concat([odf[odf.AsLOC][['Text', 'Source']],
                cdf[cdf.AsLOC][['Text', 'Source']]])

df.to_csv('data.csv', index=False)

df

,Text,Source
6,"Last week, Sen. Malcolm Wallop -LRB- R., Wyo. ...",ontonotes5
14,Rules that set standards for products or gover...,ontonotes5
17,Determining when handicapped access is require...,ontonotes5
20,"``It's very costly and time-consuming ,'' says...",ontonotes5
28,"Next to medical insurance, ``costs of complian...",ontonotes5
...,...,...
3446,""" The years I spent as manager of the Republic...",conll2003
3447,It all culminated in the fact that I now have ...,conll2003
3450,"Charlton managed Ireland for 93 matches, durin...",conll2003
3451,He guided Ireland to two successive World Cup ...,conll2003


In [57]:
[*odf.Text]

['People start their own businesses for many reasons.',
 'But a chance to fill out sales-tax records is rarely one of them.',
 'Red tape is the bugaboo of small business.',
 'Ironically, the person who wants to run his or her own business is probably the active, results-oriented sort most likely to hate meeting the rules and record-keeping demands of federal, state and local regulators.',
 'Yet every business owner has to face the mound of forms and regulations--and often is the only one available to tackle it.',
 'There is hope of change.',
 'Last week, Sen. Malcolm Wallop -LRB- R., Wyo. -RRB- held hearings on a bill to strengthen an existing law designed to reduce regulatory hassles for small businesses.',
 "``A great many federal regulations are meant for larger entities and don't really apply to small businesses ,'' says Marian Jacob, a legislative aide to Sen. Wallop.",
 'Other lawmakers are busy trying to revive the recently lapsed Paperwork Reduction Act, which many feel benefit